# Generate ML data using MOM6

# Masking can be improved. 

### Setup dask

In [1]:
# Boiler plate code to get the gateway started. 
from dask_gateway import Gateway
gateway = Gateway()

# close existing clusters
open_clusters = gateway.list_clusters()
print(list(open_clusters))

flag_shut = True 

if flag_shut: 
    if len(open_clusters)>0:
        for c in open_clusters:
            cluster = gateway.connect(c.name)
            cluster.shutdown()  

options = gateway.cluster_options()

#options.worker_memory = 16 # 24 works fine for long term mean, but seasonal needs a bit more
# options.worker_cores = 8

options.environment = dict(
    DASK_DISTRIBUTED__SCHEDULER__WORKER_SATURATION="1.0"
)

# Create a cluster with those options
cluster = gateway.new_cluster(options)
client = cluster.get_client()
cluster.adapt(15, 40)
#cluster.scale(10)
client

[]


Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/prod.64e79ff0aaa348889a604ff0da0ad58b/status,


Task exception was never retrieved
future: <Task finished name='Task-1000' coro=<Client._gather.<locals>.wait() done, defined at /srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/client.py", line 2394, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-1266' coro=<Client._gather.<locals>.wait() done, defined at /srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/client.py:2385> exception=AllExit()>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.12/site-packages/distributed/client.py", line 2394, in wait
    raise AllExit()
distributed.client.AllExit
Task exception was never retrieved
future: <Task finished name='Task-1239' coro=<Client._gather.<locals>.wait() done, defined at /srv/conda/envs/notebook/lib/pyth

In [2]:
# If wanting to look at local server.
# This allows you to see the dask dashboard for you own machine, in the
# scenario where you are not expanding out to other nodes. 
#import dask
#import distributed

#client = distributed.Client()
#client

In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from xgcm import Grid
#from xmitgcm import open_mdsdataset
import xrft
#from scipy.ndimage import gaussian_filter
import gcm_filters as gcmf

import warnings
warnings.filterwarnings('ignore')

In [3]:
from dask.diagnostics import ProgressBar

In [4]:
import os
PERSISTENT_BUCKET = os.environ['PERSISTENT_BUCKET']

In [5]:
PERSISTENT_BUCKET

'gs://leap-persistent/dhruvbalwada'

In [7]:
import gcsfs
fs = gcsfs.GCSFileSystem()
fs.ls(f'{PERSISTENT_BUCKET}')

['leap-persistent/dhruvbalwada/MOM6',
 'leap-persistent/dhruvbalwada/NN_data_Abigail',
 'leap-persistent/dhruvbalwada/SI_pred_project',
 'leap-persistent/dhruvbalwada/m2lines_transfer',
 'leap-persistent/dhruvbalwada/ssh_reconstruction_project',
 'leap-persistent/dhruvbalwada/test_offsite_upload.zarr']

In [8]:
# Open some dataset
ds = xr.open_zarr('gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/res4km_sponge10day_long.zarr')

In [9]:
ds

<xarray.Dataset> Size: 52GB
Dimensions:  (Time: 7200, zl: 2, yq: 401, xq: 301, zi: 3, yh: 400, xh: 300)
Coordinates:
  * Time     (Time) object 58kB 0001-01-11 00:00:00 ... 0198-02-16 00:00:00
  * xh       (xh) float64 2kB 2.0 6.0 10.0 14.0 ... 1.19e+03 1.194e+03 1.198e+03
  * xq       (xq) float64 2kB 0.0 4.0 8.0 12.0 ... 1.192e+03 1.196e+03 1.2e+03
  * yh       (yh) float64 3kB 2.0 6.0 10.0 14.0 ... 1.59e+03 1.594e+03 1.598e+03
  * yq       (yq) float64 3kB 0.0 4.0 8.0 12.0 ... 1.592e+03 1.596e+03 1.6e+03
  * zi       (zi) float64 24B 1.03e+03 1.032e+03 1.034e+03
  * zl       (zl) float64 16B 1.031e+03 1.033e+03
Data variables:
    PV       (Time, zl, yq, xq) float32 7GB dask.array<chunksize=(72, 2, 401, 301), meta=np.ndarray>
    RV       (Time, zl, yq, xq) float32 7GB dask.array<chunksize=(72, 2, 401, 301), meta=np.ndarray>
    e        (Time, zi, yh, xh) float32 10GB dask.array<chunksize=(72, 3, 400, 300), meta=np.ndarray>
    h        (Time, zl, yh, xh) float64 14GB dask.array<chunksize=(72, 2, 400, 300), meta=np.ndarray>
    u        (Time, zl, yh, xq) float32 7GB dask.array<chunksize=(72, 2, 400, 301), meta=np.ndarray>
    v        (Time, zl, yq, xh) float32 7GB dask.array<chunksize=(72, 2, 401, 300), meta=np.ndarray>
Attributes:
    filename:   prog__00010.nc
    grid_tile:  N/A
    grid_type:  regular
    title:      Phillips experiment

### Setup you calculations 

In [10]:
## Create grid area element 
coords = {'X': {'center': 'xh', 'outer': 'xq'},
            'Y': {'center': 'yh', 'outer': 'yq'},
            'Z': {'center': 'zl', 'outer': 'zi'} }

grid = Grid(ds, coords=coords, periodic=['X'])

dx = 1e3*grid.diff(ds.xq,'X', boundary='extend') #* np.cos(ds.yh*np.pi/180)
dy = 1e3*grid.diff(ds.yq,'Y', boundary='extend')
area_t = dy*dx

In [11]:
#Lfilter = np.array([100e3])
#dx = 4e3 

Lfilter = int(50e3/4e3)


coarsen_factor = 5 
coarsen_points= int(Lfilter/coarsen_factor)

Lfilter, coarsen_points

(12, 2)

In [12]:
def filter_dataset(ds): 

    ## parameters
    h_min = 20 #m
    dx_min = 1
    filter_scale = Lfilter # lat/lon points

    ## Make masks
    
    # make wet mask for layer thickness
    wet_mask_h = (ds.h>h_min).astype('float32').rename('wet_mask')
    # Add edges to domain
    
    wet_mask_h = wet_mask_h.where(ds.yh <1595., other=0)
    #wet_mask_h = wet_mask_h.where(ds.xh>=0.03, other=0)

    # Make wet mask for interface points, by taking the mask for the layer below.
    wet_mask_e = wet_mask_h.copy()
    wet_mask_e = wet_mask_e.rename({'zl': 'zi'}).drop_vars('zi')
    #zeros_array = xr.DataArray(np.zeros((wet_mask_e.sizes['Time'], 1, wet_mask_e.sizes['yh'], wet_mask_e.sizes['xh'])),
    #                           dims=('Time', 'zi', 'yh', 'xh'))
    zeros_array = xr.DataArray(np.zeros((wet_mask_e.sizes['Time'], 1, wet_mask_e.sizes['yh'], wet_mask_e.sizes['xh'])),
                               dims=('Time','zi', 'yh', 'xh'))
    wet_mask_e = xr.concat([wet_mask_e, zeros_array], dim='zi').chunk({'zi':-1, 'xh':-1,'yh':-1})

    # make wet masks for interface but on zl points
    wet_mask_e_up = xr.DataArray(wet_mask_e.isel(zi=slice(0, -1)).data, 
                         dims=['Time','zl','yh','xh'])
    wet_mask_e_down = xr.DataArray(wet_mask_e.isel(zi=slice(1, None)).data, 
                         dims=['Time','zl','yh','xh'])


    filter_C_up = gcmf.Filter(filter_scale= filter_scale,  
                     dx_min = dx_min, 
                     filter_shape=gcmf.FilterShape.GAUSSIAN,
                     grid_type=gcmf.GridType.REGULAR_WITH_LAND_AREA_WEIGHTED,
                     grid_vars = {'area':area_t,'wet_mask': wet_mask_e_up})

    filter_C_down = gcmf.Filter(filter_scale= filter_scale,  
                     dx_min = dx_min, 
                     filter_shape=gcmf.FilterShape.GAUSSIAN,
                     grid_type=gcmf.GridType.REGULAR_WITH_LAND_AREA_WEIGHTED,
                     grid_vars = {'area':area_t,'wet_mask': wet_mask_e_down})

    filter_C_e = gcmf.Filter(filter_scale= filter_scale,  
                     dx_min = dx_min, 
                     filter_shape=gcmf.FilterShape.GAUSSIAN,
                     grid_type=gcmf.GridType.REGULAR_WITH_LAND_AREA_WEIGHTED,
                     grid_vars = {'area':area_t,'wet_mask': wet_mask_e})

    ## Filter

    # Center velocities
    u_c = grid.interp(ds.u.fillna(0), 'X', boundary='extend')
    v_c = grid.interp(ds.v.fillna(0), 'Y', boundary='extend')

    # Bring interface values to zl, so they can be operated on with velocities
    e_up = xr.DataArray(ds.e.isel(zi=slice(0, -1)).data, 
                         dims=['Time','zl','yh','xh'])
    e_down = xr.DataArray(ds.e.isel(zi=slice(1, None)).data, 
                         dims=['Time','zl','yh','xh'])

    ebar_up = filter_C_up.apply(e_up, dims=['yh','xh'])
    ebar_down = filter_C_down.apply(e_down, dims=['yh','xh'])
    
    hbar = ebar_up - ebar_down 

    ebar = filter_C_e.apply(ds.e, dims=['yh','xh'])
    
    # u component
    ue_bar_up = filter_C_up.apply(u_c * e_up, dims=['yh','xh'])
    ue_bar_down = filter_C_down.apply(u_c * e_down, dims=['yh','xh'])

    ubar_up = filter_C_up.apply(u_c, dims=['yh','xh'])
    ubar_down = filter_C_down.apply(u_c, dims=['yh','xh'])
    ubar = ubar_up # mask for upper interface is same as that for a layer (if layer vanishes then interface is grounded).
    
    ubar_up_ebar_up =   ubar_up * ebar_up
    ubar_down_ebar_down =   ubar_down * ebar_down

    uh_bar = ue_bar_up - ue_bar_down
    ubar_hbar = ubar_up_ebar_up - ubar_down_ebar_down 

    uphp = uh_bar - ubar_hbar

    # v component
    ve_bar_up = filter_C_up.apply(v_c * e_up, dims=['yh','xh'])
    ve_bar_down = filter_C_down.apply(v_c * e_down, dims=['yh','xh'])

    vbar_up = filter_C_up.apply(v_c, dims=['yh','xh'])
    vbar_down = filter_C_down.apply(v_c, dims=['yh','xh'])
    vbar = vbar_up
    
    vbar_up_ebar_up = vbar_up * ebar_up
    vbar_down_ebar_down = vbar_down * ebar_down

    vh_bar = ve_bar_up - ve_bar_down
    vbar_hbar = vbar_up_ebar_up - vbar_down_ebar_down 

    vphp = vh_bar - vbar_hbar

    # Add ubar, vbar, hbar, ebar, uphp, vphp to a dataset
    ds_filt = xr.Dataset(coords=ds.coords)
    ds_filt['ubar'] = ubar
    ds_filt['vbar'] = vbar
    ds_filt['hbar'] = hbar
    ds_filt['ebar'] = ebar
    ds_filt['uphp'] = uphp
    ds_filt['vphp'] = vphp
    
    return ds_filt


def add_gradients(ds_filt): 
    ds_filt['dudx'] = grid.interp(grid.diff(ds_filt.ubar, 'X'), 'X')/dx
    ds_filt['dvdx'] = grid.interp(grid.diff(ds_filt.vbar, 'X'), 'X')/dx
    ds_filt['dudy'] = grid.interp(grid.diff(ds_filt.ubar, 'Y'), 'Y')/dy
    ds_filt['dvdy'] = grid.interp(grid.diff(ds_filt.vbar, 'Y'), 'Y')/dy
    
    ds_filt['dedx'] = grid.interp(grid.diff(ds_filt.ebar, 'X'), 'X')/dx
    ds_filt['dedy'] = grid.interp(grid.diff(ds_filt.ebar, 'Y'), 'Y')/dy

    ds_filt['dhdx'] = grid.interp(grid.diff(ds_filt.hbar, 'X'), 'X')/dx
    ds_filt['dhdy'] = grid.interp(grid.diff(ds_filt.hbar, 'Y'), 'Y')/dy

    return ds_filt

def coarsen_dataset(ds, coarsen_points): 
    print('coarsening')
    return ds.coarsen(xh=coarsen_points, yh=coarsen_points, boundary='trim').mean()

In [13]:
# def filter_dataset(ds, Lfilter): 
    
#     dx = ds.xh.diff('xh')[0].values*1e3
#     #wet_mask= (ds.h.isel(Time=0, zl=0)*0. + 1.).rename('wet_mask')
#     #wet_mask_w_land = wet_mask
#     wet_mask = (ds.h.isel(Time=0, zl=1)>1).astype('float32').rename('wet_mask')
#     wet_mask_w_land = wet_mask.where(ds.yh <1595., other=0)
    
#     filter_C = gcmf.Filter(filter_scale= Lfilter,  
#                      dx_min = dx, 
#                      filter_shape=gcmf.FilterShape.GAUSSIAN,
#                      grid_type=gcmf.GridType.REGULAR_WITH_LAND,
#                      grid_vars = {'wet_mask': wet_mask_w_land})
    
#     xgrid = Grid(ds, coords={'X': {'center': 'xh', 'outer': 'xq'},
#                          'Y': {'center': 'yh', 'outer': 'yq'},
#                          'Z': {'center': 'zl', 'outer': 'zi'} },
#              periodic=['X'])
    
#     ds_filt = xr.Dataset() # For storing the filtered data 
#     print('Started')
#     ds_filt['h'] = filter_C.apply(ds.h, dims=['yh','xh']).rename('h')
#     ds_filt['e'] = filter_C.apply(ds.e, dims=['yh','xh']).rename('e')
#     u_c = xgrid.interp(ds.u.fillna(0), 'X')
#     v_c = xgrid.interp(ds.v.fillna(0), 'Y')
#     ds_filt['u'] = filter_C.apply(u_c, dims=['yh','xh']).rename('u')
#     ds_filt['v'] = filter_C.apply(v_c, dims=['yh','xh']).rename('v')
    
#     print('Filtered fields computed')
    
#     ds_filt['uh'] = ds.h*u_c
#     ds_filt['vh'] = ds.h*v_c
    
#     ds_filt['uu'] = u_c*u_c
#     ds_filt['vv'] = v_c*v_c
    
#     ds_filt['uh_bar'] = filter_C.apply(ds_filt['uh'], dims=['yh','xh']).rename('uh_bar')
#     ds_filt['vh_bar'] = filter_C.apply(ds_filt['vh'], dims=['yh','xh']).rename('vh_bar')
    
#     ds_filt['uu_bar'] = filter_C.apply(ds_filt['uu'], dims=['yh','xh']).rename('uu_bar')
#     ds_filt['vv_bar'] = filter_C.apply(ds_filt['vv'], dims=['yh','xh']).rename('vv_bar')
    
#     ds_filt['ubar_hbar'] = (ds_filt.h*ds_filt.u)
#     ds_filt['vbar_hbar'] = (ds_filt.h*ds_filt.v)
    
#     ds_filt['ubar_ubar'] = (ds_filt.u*ds_filt.u)
#     ds_filt['vbar_vbar'] = (ds_filt.v*ds_filt.v)
    
#     ds_filt['uh_sg'] = ds_filt['uh_bar'] - ds_filt['ubar_hbar']
#     ds_filt['vh_sg'] = ds_filt['vh_bar'] - ds_filt['vbar_hbar']
    
#     ds_filt['uu_sg'] = ds_filt['uu_bar'] - ds_filt['ubar_ubar']
#     ds_filt['vv_sg'] = ds_filt['vv_bar'] - ds_filt['vbar_vbar']
    
#     print('Fluxes computed')
    
#     ds_filt['dudx'] = xgrid.interp(xgrid.diff(ds_filt.u, 'X')/dx, 'X')
#     ds_filt['dvdx'] = xgrid.interp(xgrid.diff(ds_filt.v, 'X')/dx, 'X')
#     ds_filt['dudy'] = xgrid.interp(xgrid.diff(ds_filt.u, 'Y')/dx, 'Y')
#     ds_filt['dvdy'] = xgrid.interp(xgrid.diff(ds_filt.v, 'Y')/dx, 'Y')
#     ds_filt['slope_x'] = xgrid.interp(xgrid.diff(ds_filt.e, 'X')/dx, 'X')
#     ds_filt['slope_y'] = xgrid.interp(xgrid.diff(ds_filt.e, 'Y')/dx, 'Y')
    
#     print('Gradients computed')
    
#     return ds_filt

In [14]:
ds_filt  = add_gradients(filter_dataset(ds))

In [15]:
ds_filt_coarsen = coarsen_dataset(ds_filt, coarsen_points) 

coarsening


In [16]:
# This the final object I want to load up.
ds_filt_coarsen = ds_filt_coarsen.drop(['xq', 'yq']).chunk({'Time':36, 'xh':-1,'yh':-1,'zl':-1,'zi':-1})

In [49]:
#ds_filt_coarsen_small = ds_filt_coarsen.isel(Time=slice(0, 1440))

In [50]:
# This will work with reasonabe number of chunks. 
# Try this first. 
#with ProgressBar():
#    ds_filt_coarsen_small.load()

In [17]:
fname = f'{PERSISTENT_BUCKET}/MOM6/Phillips2Layer/test_res4km_sponge10day_long_ml_data_'+str(int(Lfilter*4e3/1e3))+'km_8_Aug_24.zarr'
fname
mapper = fs.get_mapper(fname)

In [18]:
fname

'gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/test_res4km_sponge10day_long_ml_data_48km_8_Aug_24.zarr'

In [19]:
# This function allows you to split the computation over parts. 
def to_zarr_split(ds, mapper, split_dim="time", split_interval=1):
    print(f"Writing to {mapper.root} ...")

    n = len(ds[split_dim])
    splits = list(range(0, n, split_interval))

    # Make sure the last item in the list covers the full length
    # of the time on our dataset
    if splits[-1] != n:
        splits = splits + [n]

    split_datasets = []
    for ii in range(len(splits) - 1):
        start = splits[ii]
        stop = splits[ii + 1]
        split_datasets.append(ds.isel({split_dim: slice(start, stop)}))

    # write the first array
    # TODO: move the first write to the loop so it is counted in the viz bar
    split_datasets[0].to_zarr(mapper)
    for ds_split in tqdm(split_datasets[1:None]):
        ds_split.to_zarr(mapper, append_dim=split_dim)

    # overwrite the split dimension as single chunk (this should reproduce
    # what xr.to_zarr would do
    g = zarr.open_group(mapper)
    del g[split_dim]
    
    ds[[split_dim]].load().to_zarr(mapper, mode='a')
    zarr.consolidate_metadata(mapper)

In [20]:
import zarr
from tqdm.auto import tqdm

In [21]:
%%time 
to_zarr_split(ds_filt_coarsen, mapper, split_dim='Time', split_interval=720)

Writing to leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/test_res4km_sponge10day_long_ml_data_48km_8_Aug_24.zarr ...


KeyboardInterrupt: 

In [ ]:
#fname = f'{PERSISTENT_BUCKET}/MOM6/Phillips2Layer/res4km_sponge10day/ml_data_'+str(int(Lfilter/1e3))+'km.zarr'
#fname = f'{PERSISTENT_BUCKET}/MOM6/Phillips2Layer/res4km_sponge60day/ml_data_'+str(int(Lfilter/1e3))+'km.zarr'
#fname = f'{PERSISTENT_BUCKET}/MOM6/Phillips2Layer/res4km_sponge300day/ml_data_'+str(int(Lfilter/1e3))+'km.zarr'


In [25]:
fname

'gs://leap-persistent/dhruvbalwada/MOM6/Phillips2Layer/res4km_sponge10day_long_ml_data_48km_8_Aug_24.zarr'

In [26]:
test = xr.open_zarr(fname)

In [27]:
test

<xarray.Dataset> Size: 42GB
Dimensions:  (Time: 7200, zi: 3, yh: 200, xh: 150, zl: 2)
Coordinates:
  * Time     (Time) object 58kB 0001-01-11 00:00:00 ... 0198-02-16 00:00:00
  * xh       (xh) float64 1kB 4.0 12.0 20.0 ... 1.18e+03 1.188e+03 1.196e+03
  * yh       (yh) float64 2kB 4.0 12.0 20.0 ... 1.58e+03 1.588e+03 1.596e+03
  * zi       (zi) float64 24B 1.03e+03 1.032e+03 1.034e+03
  * zl       (zl) float64 16B 1.031e+03 1.033e+03
Data variables: (12/14)
    dedx     (Time, zi, yh, xh) float64 5GB dask.array<chunksize=(36, 3, 200, 150), meta=np.ndarray>
    dedy     (Time, zi, yh, xh) float64 5GB dask.array<chunksize=(36, 3, 200, 150), meta=np.ndarray>
    dhdx     (Time, zl, yh, xh) float64 3GB dask.array<chunksize=(36, 2, 200, 150), meta=np.ndarray>
    dhdy     (Time, zl, yh, xh) float64 3GB dask.array<chunksize=(36, 2, 200, 150), meta=np.ndarray>
    dudx     (Time, zl, yh, xh) float64 3GB dask.array<chunksize=(36, 2, 200, 150), meta=np.ndarray>
    dudy     (Time, zl, yh, xh) float64 3GB dask.array<chunksize=(36, 2, 200, 150), meta=np.ndarray>
    ...       ...
    ebar     (Time, zi, yh, xh) float32 3GB dask.array<chunksize=(36, 3, 200, 150), meta=np.ndarray>
    hbar     (Time, zl, yh, xh) float32 2GB dask.array<chunksize=(36, 2, 200, 150), meta=np.ndarray>
    ubar     (Time, zl, yh, xh) float32 2GB dask.array<chunksize=(36, 2, 200, 150), meta=np.ndarray>
    uphp     (Time, zl, yh, xh) float32 2GB dask.array<chunksize=(36, 2, 200, 150), meta=np.ndarray>
    vbar     (Time, zl, yh, xh) float32 2GB dask.array<chunksize=(36, 2, 200, 150), meta=np.ndarray>
    vphp     (Time, zl, yh, xh) float32 2GB dask.array<chunksize=(36, 2, 200, 150), meta=np.ndarray>

In [59]:
#fs.ls(f'{PERSISTENT_BUCKET}/MOM6/Phillips2Layer/res4km_sponge300day/')

### When done

In [28]:
cluster.shutdown()